<a href="https://colab.research.google.com/github/Siddharthareddy25/IoT-Based-Environmental-Monitoring-System-/blob/main/temp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input
from sklearn.ensemble import RandomForestRegressor
import warnings

warnings.filterwarnings("ignore")

df = pd.read_csv('temp_data.csv', encoding='latin1')

df['time'] = pd.to_datetime(df['time'])
df['hour'] = df['time'].dt.hour

df = df[['temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour', 'precipitation_probability (%)']]

scaler_temp = MinMaxScaler()
scaler_hum = MinMaxScaler()
df[['temperature_2m (°C)']] = scaler_temp.fit_transform(df[['temperature_2m (°C)']])
df[['relative_humidity_2m (%)']] = scaler_hum.fit_transform(df[['relative_humidity_2m (%)']])

def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
    return np.array(X), np.array(y)

n_steps = 24

temp_series = df['temperature_2m (°C)'].values
X_temp, y_temp = create_sequences(temp_series, n_steps)
X_temp = X_temp.reshape((X_temp.shape[0], X_temp.shape[1], 1))

model_temp = Sequential([
    Input(shape=(n_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])
model_temp.compile(optimizer='adam', loss='mse')
model_temp.fit(X_temp, y_temp, epochs=20, verbose=0)

hum_series = df['relative_humidity_2m (%)'].values
X_hum, y_hum = create_sequences(hum_series, n_steps)
X_hum = X_hum.reshape((X_hum.shape[0], X_hum.shape[1], 1))

model_hum = Sequential([
    Input(shape=(n_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])
model_hum.compile(optimizer='adam', loss='mse')
model_hum.fit(X_hum, y_hum, epochs=20, verbose=0)

df_precip = df.dropna()
X_precip = df_precip[['temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour']]
y_precip = df_precip['precipitation_probability (%)']
precip_model = RandomForestRegressor(n_estimators=100, random_state=42)
precip_model.fit(X_precip, y_precip)
print("Precipitation model trained successfully.")

present_temp = float(input("Enter the current temperature (°C): "))
present_hum = float(input("Enter the current humidity (%): "))
present_hour = int(input("Enter the current hour (0-23): "))

present_temp_df = pd.DataFrame([[present_temp]], columns=['temperature_2m (°C)'])
present_hum_df = pd.DataFrame([[present_hum]], columns=['relative_humidity_2m (%)'])
scaled_present_temp = scaler_temp.transform(present_temp_df)[0][0]
scaled_present_hum = scaler_hum.transform(present_hum_df)[0][0]

last_temp_seq = temp_series[-(n_steps-1):].tolist()
last_temp_seq.append(scaled_present_temp)
last_temp_seq = np.array(last_temp_seq).reshape((1, n_steps, 1))

last_hum_seq = hum_series[-(n_steps-1):].tolist()
last_hum_seq.append(scaled_present_hum)
last_hum_seq = np.array(last_hum_seq).reshape((1, n_steps, 1))

pred_temp = model_temp.predict(last_temp_seq, verbose=0)
pred_temp_real = scaler_temp.inverse_transform([[pred_temp[0][0]]])[0][0]

pred_hum = model_hum.predict(last_hum_seq, verbose=0)
pred_hum_real = scaler_hum.inverse_transform([[pred_hum[0][0]]])[0][0]

next_hour = (present_hour + 1) % 24
pred_temp_df = pd.DataFrame([[pred_temp_real]], columns=['temperature_2m (°C)'])
pred_hum_df = pd.DataFrame([[pred_hum_real]], columns=['relative_humidity_2m (%)'])
pred_scaled_temp = scaler_temp.transform(pred_temp_df)[0][0]
pred_scaled_hum = scaler_hum.transform(pred_hum_df)[0][0]

input_data = pd.DataFrame([[pred_scaled_temp, pred_scaled_hum, next_hour]],
                          columns=['temperature_2m (°C)', 'relative_humidity_2m (%)', 'hour'])

pred_precip = precip_model.predict(input_data)[0]

print(f"Predicted temperature 1 hour later: {pred_temp_real:.2f} °C")
print(f"Predicted humidity 1 hour later: {pred_hum_real:.2f} %")
print(f"Predicted precipitation probability: {pred_precip:.2f} %")

Precipitation model trained successfully.
Enter the current temperature (°C): 35
Enter the current humidity (%): 70
Enter the current hour (0-23): 13
Predicted temperature 1 hour later: 31.84 °C
Predicted humidity 1 hour later: 72.05 %
Predicted precipitation probability: 7.83 %
